In [1]:
from probttrtypes import Type, PConstraint
from utils import show, show_latex

The witness cache in `probttr` is a pair whose first member is a list of objects and whose second member is a list of probabilities (actually probability constraints)

In [2]:
T = Type()
print(T.witness_cache)

([], [])


In [3]:
show(T.judge('a',.5))

'0.5'

In [4]:
show(T.witness_cache)

'([a], [0.5])'

In [5]:
show(T.judge('a',.6))

'0.6'

In [6]:
show(T.witness_cache)

'([a], [0.6])'

Adding an additional probability argument to `judge()` gives you a minimum and maximum probability constraint.

In [7]:
show(T.judge('a',.6,1))

'>=0.6'

In [8]:
show_latex(T.judge('a',.6,.8))

<IPython.core.display.Latex object>

In [9]:
show_latex(T.judge('a',0,.6))

<IPython.core.display.Latex object>

`judge(a,n,n)` is the same as `judge(a,n)`

In [10]:
show_latex(T.judge('a',.6,.6))

<IPython.core.display.Latex object>

Probabilities must be between 0 and 1.  For example, `show(T.judge('a',-1))` and `show(T.judge('a',.6,.1))` raise exceptions.

In contrast to non-probabilistic `pyttr` we can store a negative judgement in witness cache.

In [12]:
T.judge('a',0)
show_latex(T.witness_cache)

<IPython.core.display.Latex object>

In [ ]:
import random
random.uniform(0.9999999999999999,1)

In [ ]:
isinstance(1.0,float)

In [ ]:
1>=0

In [ ]:
None>1